In [5]:
from model import *
import pyomo.environ as pyo
from unit_models.valve import SVValve
from unit_models.pid_controller import SVPIDController
from unit_models.heater import SVHeater
from idaes.core import FlowsheetBlock, MaterialBalanceType
from idaes.models.properties import iapws95
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.solvers import get_solver
from idaes.models.control.controller import (
    ControllerType,
    ControllerMVBoundType
)
from idaes.core.util import DiagnosticsToolbox
import idaes.logger as idaeslog

def _add_inlet_pressure_step(m, time=1, value=6.0e5):
    """Add an inlet pressure step change"""
    for t in m.fs.time:
        if t >= time:
            m.fs.valve_1.inlet.pressure[t].fix(value)




def create_model(time_set=None, time_units=pyo.units.s, nfe=5, tee=False):
    """Build and initialize the flowsheet model

           valve_1   +----+
  steam ----|><|-->--| ta |    valve_2
                     | nk |-----|><|--->--- steam
                     +----+
    """
    m = pyo.ConcreteModel(name="Dynamic Steam Tank with PID Control")
    # Add flowsheet
    m.fs = FlowsheetBlock(dynamic=True, time_set=time_set, time_units=time_units)
    # Add water property parameter block
    m.fs.prop_water = iapws95.Iapws95ParameterBlock(
        phase_presentation=iapws95.PhaseType.LG
    )
    # Add valve 1
    m.fs.valve_1 = SVValve(
        dynamic=False,
        has_holdup=False,
        material_balance_type=MaterialBalanceType.componentTotal,
        property_package=m.fs.prop_water,
    )
    # Add heater model to represent a tank (close to bare control volume model).
    m.fs.tank = SVHeater(
        has_holdup=True,
        material_balance_type=MaterialBalanceType.componentTotal,
        property_package=m.fs.prop_water,
    )
    # Add valve 2
    m.fs.valve_2 = SVValve(
        dynamic=False,
        has_holdup=False,
        material_balance_type=MaterialBalanceType.componentTotal,
        property_package=m.fs.prop_water,
    )
    # Add a controller
    m.fs.ctrl = SVPIDController(
        process_var=m.fs.tank.control_volume.properties_out[:].pressure,
        manipulated_var=m.fs.valve_1.valve_opening,
        calculate_initial_integral=True,
        mv_bound_type=ControllerMVBoundType.SMOOTH_BOUND,
        controller_type=ControllerType.PI,
    )
    # The control volume block doesn't assume equilibrium, so I'll make that
    # assumption here. I don't actually expect liquid to form but who knows?
    # The phase_fraction in the control volume is volumetric phase fraction.
    @m.fs.tank.Constraint(m.fs.time)
    def vol_frac_vap(b, t):
        return (
            b.control_volume.properties_out[t].phase_frac["Vap"]
            * b.control_volume.properties_out[t].dens_mol
            / b.control_volume.properties_out[t].dens_mol_phase["Vap"]
        ) == (b.control_volume.phase_fraction[t, "Vap"])

    # Connect the models
    m.fs.v1_to_tank = Arc(source=m.fs.valve_1.outlet, destination=m.fs.tank.inlet)
    m.fs.tank_to_v2 = Arc(source=m.fs.tank.outlet, destination=m.fs.valve_2.inlet)

    
    # Add the stream constraints
    pyo.TransformationFactory("network.expand_arcs").apply_to(m.fs)

    # # Register inlet ports for the model so state var replacement works
    # pprint_replacements(m.fs)
    

    # Do DAE discretization
    pyo.TransformationFactory("dae.finite_difference").apply_to(
        m.fs,
        nfe=nfe,
        wrt=m.fs.time,
        scheme="BACKWARD"
    )

    register_inlet_ports(m.fs)


    # Fix the derivative variables to zero at time 0 (steady state assumption)
    # m.fs.fix_initial_conditions()


    m.fs.ctrl.mv_lb = 0.0 # valve opening lower bound is 0. Note this isn't a variable, its' just a constant. so it doesn't show up in state vars.
    m.fs.ctrl.mv_ub = 1.0 # value opening upper bound is 1

    # Replacements
    #replace_state_var(m.fs.valve_1.inlet.flow_mol, m.fs.valve_2.outlet.pressure)
    # m.fs.valve_1.deltaP.fix()
    # m.fs.valve_2.deltaP.fix()
    # m.fs.valve_1.valve_opening.unfix()
    # m.fs.valve_2.valve_opening.unfix() 
    # replace_state_var(m.fs.valve_1.deltaP, m.fs.valve_1.valve_opening)
    # replace_state_var(m.fs.valve_2.deltaP, m.fs.valve_2.valve_opening)

    m.fs.ctrl.setpoint.fix(3e5) # setpoint is tank pressure of 300 kPa
    #m.fs.valve_2.outlet.pressure.fix(101325) # valve 2 outlet pressure
    m.fs.valve_1.inlet.flow_mol.fix(100) # valve 1 inlet molar flowrate

    m.fs.fix_initial_conditions()
    # Fix the input variables
    m.fs.valve_1.Cv.fix(0.9) # valve 1 flow coefficient
    m.fs.valve_1.inlet.enth_mol.fix(50000) # inlet fluid molar enthalpy
    m.fs.valve_1.inlet.pressure.fix(5e5) # inlet pressure of valve 1
    m.fs.tank.heat_duty.unfix()
    m.fs.tank.heat_duty.fix(0) # no heat transfer to tank
    m.fs.tank.control_volume.volume.fix(2.0) # 2 m^3 tank volume
    props0 = m.fs.tank.control_volume.properties_in[0]
    #props0.pressure.fix(5e5)
    #props0.enth_mol.fix(50000)
    props0.flow_mol.fix(100)
    #m.fs.tank.control_volume.phase_fraction[0, "Liq"].fix(1.0) # assume all vapor in tank
    #m.fs.tank.outlet.flow_mol[0].fix(100) # tank outlet flowrate at time 0
    # m.fs.tank.control_volume.energy_accumulation[0, "Liq"].fix(0)
    # m.fs.tank.control_volume.energy_accumulation[0, "Vap"].fix(0)
    # m.fs.tank.control_volume.material_accumulation[0,"Liq","H2O"].fix(0)
    # m.fs.tank.control_volume.material_accumulation[0,"Vap","H2O"].fix(0)
    m.fs.valve_2.Cv.fix(2) # valve 2 flow coefficient
    m.fs.valve_2.valve_opening.fix(1) # fix valve 2 opening to full open
    m.fs.ctrl.gain_p.fix(1e-6) # proportional gain
    m.fs.ctrl.gain_i.fix(1e-5) # integral error gain
    m.fs.ctrl.mv_ref.fix(0) # controller bias

    print("=== After Fixing and Replacing ===")
    pprint_replacements(m.fs)
    print("degrees of freedom", degrees_of_freedom(m))

    #m.fs.valve_1.valve_opening.fix(1) # fix valve 1 opening to full open
    # use model diagnostics to find overconstrained set
    dt = DiagnosticsToolbox(m)
    dt.display_overconstrained_set()
    dt.display_underconstrained_set()

    # Initialize the model
    solver = get_solver(options={"max_iter": 50})
    m.fs.valve_1.initialize(outlvl=idaeslog.INFO)
    m.fs.valve_1.report()
    propagate_state(m.fs.v1_to_tank)
    m.fs.tank.initialize(outlvl=idaeslog.INFO)
    m.fs.tank_1.report()
    propagate_state(m.fs.tank_to_v2)
    m.fs.valve_2.initialize(outlvl=idaeslog.INFO)
    m.fs.valve_2.report()


    assert degrees_of_freedom(m) == 0


    # Return the model and solver
    return m, solver


# Create a model for the 0 to 12 sec time period
m, solver = create_model(time_set=[0, 3], nfe=3, tee=False)
print("Solving the model...")
solver.solve(m, tee=True)

=== After Fixing and Replacing ===
Replacements in block fs:
(Variable -> Replaced State Var)
  fs.ctrl.setpoint -> fs.valve_1.valve_opening

Unreplaced state variables in block fs:
  fs.valve_1.Cv
  fs.valve_1._flow_mol_inlet_ref
  fs.valve_1._enth_mol_inlet_ref
  fs.valve_1._pressure_inlet_ref
  fs.tank.heat_duty
  fs.tank.control_volume.volume
  fs.tank.control_volume.energy_accumulation[0.0,Liq]
  fs.tank.control_volume.energy_accumulation[0.0,Vap]
  fs.valve_2.valve_opening
  fs.valve_2.Cv
  fs.ctrl.gain_p
  fs.ctrl.gain_i
  fs.ctrl.mv_ref
degrees of freedom 0
Dulmage-Mendelsohn Over-Constrained Set

Dulmage-Mendelsohn Under-Constrained Set

2025-11-18 11:28:34 [INFO] idaes.init.fs.valve_1: Initialization Complete: optimal - Optimal Solution Found

Unit : fs.valve_1                                                          Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key               : Val

AttributeError: '_ScalarFlowsheetBlock' object has no attribute 'tank_1'